In [34]:
import numpy as np
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from pylab import rcParams
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
%load_ext dotenv
%dotenv ../.env

In [3]:
cnx = create_engine(os.getenv('EDMDB'))

In [9]:
facilities = pd.read_sql_query('''SELECT f.* FROM dcp.facilities f;''', cnx)
facilities.head()

,id,geom,uid,facname,addressnum,streetname,address,city,zipcode,boro,...,overlevel,capacity,captype,proptype,latitude,longitude,xcoord,ycoord,datasource,facsubgrp
0,1349,0101000020D7080000099B2EDC719B2E418D8275E22E19...,4f384e2e29509d47137ceda6ebf6530f,ADJ BRONX TERMINAL MARKET,None,None,None,None,None,Manhattan,...,City,None,None,City Owned,40.8209373825321,-73.932483746857,1002937,238373.999999957,dcas_colp,MISCELLANEOUS USE
1,10891,0101000020D70800000C030056A4962F41CA4A34843A37...,d491ec4dabb06564040e0e01cc10c06a,EVERS,None,EVERS,EVERS,BRONX,10465,Bronx,...,Federal,None,None,None,40.845933611146,-73.816244721778,1035090.24524262,247527.173040469,usdot_airports,AIRPORTS AND HELIPORTS
2,2209,0101000020D7080000298D6C903A352F4131C1ECB35E33...,959f085e939096e88692ed8cbddf823d,"ARMSTRONG, LOUIS, HOUSE",None,None,None,CORONA,11368,Queens,...,State,None,None,None,40.7557191956,-73.8615,1022621.28196551,214635.90911726,nysparks_historicplaces,HISTORICAL SITES
3,19564,0101000020D7080000F321FF20C2F92F419550CD313097...,b4c74cc92ed8691e51aaafdfe5e3f580,NEW YORK CITY D.O.T. HART ISLAND FERRY SLIP,None,HART ISLAND,None,BRONX,10464,Bronx,...,City,None,None,None,40.85428,-73.770361,1047777.06444651,250598.024317388,usdot_ports,PORTS AND FERRY LANDINGS
4,23472,0101000020D7080000B1679A00D0042E41A441F7302967...,6bd2ce56405e06811d0e1c5f6692e3e3,PIER 6,None,None,PIER 6,None,None,Manhattan,...,City,None,None,City Owned,40.6928955912787,-74.0021420042349,983655.999999999,191716.999999942,dcas_colp,MISCELLANEOUS USE


Some of the BBLs on the facilities table are null. Could they be backfilled using a POSTGIS query such as this one?

In [18]:
bbls_for_nulls = pd.read_sql_query('''SELECT CAST(p."BBL" AS TEXT) AS pluto_bbl, f.bbl, f.facname, f.factype
FROM dcp.facilities f, dcp.pluto202 p
WHERE f.bbl IS NULL
AND ST_Within(f.geom, p.geom)''', cnx)
bbls_for_nulls

,pluto_bbl,bbl,facname,factype
0,5044440001,None,VOORLEZER'S HOUSE,STATE HISTORIC PLACE
1,1007297503,None,"450 WEST 33 STREET, NEW YORK, NY 10001",PRIVATELY OWNED PUBLIC SPACE
2,5013010001,None,ARLINGTON MARSH PARK,UNDEVELOPED
3,3023487501,None,"AUSTIN, NICHOLS & COMPANY WAREHOUSE",STATE HISTORIC PLACE
4,2056500001,None,BARTOW-PELL MANSION AND CARRIAGE HOUSE,STATE HISTORIC PLACE
...,...,...,...,...
70,5023590001,None,WILLOWBROOK PARKWAY,PARKWAY
71,4142600001,None,TRANS WORLD AIRLINES FLIGHT CENTER,STATE HISTORIC PLACE
72,4081620097,None,UDALL'S COVE AND RAVINE NATURAL RESOURCE AREA,NATURAL RESOURCE AREA
73,4081620097,None,UDALL'S PARK PRESERVE,NATURE AREA


In [14]:
factype = facilities.groupby("bbl")["factype"] \
                            .count() \
                            .reset_index(name='count') \
                            .sort_values(['count'], ascending=False)
factype = factype[factype["count"] > 1]
factype.head()

,bbl,count
1311,1007860001,39
2736,1013730001,30
4929,2026050040,28
14074,4096480001,25
1819,1009620100,25


Identify Factype duplicates?

In [23]:
potential_factype_dups = pd.read_sql_query('''WITH potential_dups AS (
	SELECT bbl, factype, count(*)
	FROM dcp.facilities f
	WHERE bbl IS NOT NULL
	GROUP BY bbl, factype
	HAVING COUNT(*) > 1
	ORDER BY COUNT(*) DESC
)
SELECT f.bbl, f.facdomain, f.facgroup, f.facsubgrp, f.factype, f.facname, f.overagency, f.address, f.id
FROM dcp.facilities f, potential_dups d
WHERE f.bbl = d.bbl
AND f.factype = d.factype
ORDER BY f.bbl, f.facdomain, f.facgroup, f.facsubgrp, f.factype;''', cnx)
potential_factype_dups.to_csv("../output/potential_factype_dups.csv", index=False)

Look for similar facility names. Looking for IDs that are close to each other seems to produce a more focused result set

In [37]:
similar_facnames = pd.read_sql_query('''SELECT f1.id, f1.facname, f1.overagency, f2.id, f2.facname, f2.overagency
FROM dcp.facilities f1, dcp.facilities f2
WHERE f1.bbl = f2.bbl
AND f1.geom = f2.geom
AND f1.address = f2.address
AND f1.facdomain = f2.facdomain
AND f1.facgroup = f2.facgroup
AND f1.facsubgrp = f2.facsubgrp
AND f1.factype = f2.factype
AND f1.facname <> f2.facname
AND f1.overagency = f2.overagency
AND ABS(f1.id - f2.id) = 1;''', cnx)
similar_facnames.to_csv('../output/similar_facnames.csv', index=False)

Get rows where commas are not followed by spaces.

In [33]:
comma_no_space = pd.read_sql_query('''SELECT * from dcp.facilities 
WHERE SUBSTRING(facname, (POSITION(',' IN facname) + 1), 1) <> ' '
AND facname LIKE '%%,%%';''', cnx)
comma_no_space.head()

,id,geom,uid,facname,addressnum,streetname,address,city,zipcode,boro,...,overlevel,capacity,captype,proptype,latitude,longitude,xcoord,ycoord,datasource,facsubgrp
0,1133,0101000020D70800008B69D7ED2AB02C41B6E50BE8F98F...,61d61686681b70042bd6afd10e7785f7,"ABC READY,SET, LEARN, INC.",315,NETHERLAND AVENUE,315 NETHERLAND AVENUE,STATEN ISLAND,10303,Staten Island,...,City,None,None,None,40.6289153266982,-74.1592234245274,940053.34691182,168447.357542829,nysed_activeinstitutions,PRESCHOOLS FOR STUDENTS WITH DISABILITIES
1,1329,0101000020D70800008195C4163AD72E417A7BE03499CC...,23eced6190179f2b2c6aa90c346b29f9,"ADAPTIVE SOLUTIONS MULTI SERVICES (ST,OT, PT, ...",185,HEGEMAN AVENUE,185 HEGEMAN AVENUE,BROOKLYN,11212,Brooklyn,...,City,None,None,None,40.6567958964744,-73.9050705864543,1010589.15925197,178579.113225909,dohmh_daycare,DAY CARE
2,3409,0101000020D7080000A0FA0BBA3E352F41500DFA97786A...,32b7d1ccf9ca1dec09df3aefbd2b20b6,"BIRCH FAMILY SERVICES,INC.",1880,WATSON AVENUE,1880 WATSON AVENUE,BRONX,10472,Bronx,...,City,None,None,None,40.8280134313224,-73.8613420655131,1022623.34499327,240975.231325072,dohmh_daycare,DAY CARE
3,3506,0101000020D70800007FD32BFDFED62E4135A2EC3D92D6...,d91fd1a5f485c1e417dcbd9cff200c22,"BLENDED FAMILY,LLC",810,EAST 152 STREET,810 EAST 152 STREET,BRONX,10455,Bronx,...,City,None,None,None,40.8150671009294,-73.9049516271841,1010559.59763131,236242.317120635,doe_universalprek,DOE UNIVERSAL PRE-KINDERGARTEN
4,4875,0101000020D70800003D0F3B441D5A2E414A9AE85F05A1...,376f3aad5538734dd26770c7352ba883,BROOKLYN SECTION NATIONAL COUNCIL OF JEWISH WO...,1001,QUENTIN ROAD,1001 QUENTIN ROAD,BROOKLYN,11223,Brooklyn,...,City,None,None,None,40.6080314990898,-73.9628156448455,994574.731799274,160800.853703582,dfta_contracts,SENIOR SERVICES


Which agencies tend to send these?

In [38]:
sending_agencies = pd.read_sql_query('''SELECT f1.overagency, count(*)
FROM dcp.facilities f1, dcp.facilities f2
WHERE f1.bbl = f2.bbl
AND f1.geom = f2.geom
AND f1.address = f2.address
AND f1.facdomain = f2.facdomain
AND f1.facgroup = f2.facgroup
AND f1.facsubgrp = f2.facsubgrp
AND f1.factype = f2.factype
AND f1.facname <> f2.facname
AND f1.overagency = f2.overagency
AND ABS(f1.id - f2.id) = 1
group by f1.overagency
order by count(*) desc;''', cnx)
sending_agencies

,overagency,count
0,NYS Office of Mental Health,104
1,NYS Office of Alcoholism and Substance Abuse S...,92
2,NYC Department of Education,88
3,NYC Department of Health and Mental Hygiene,66
4,NYC Department of Youth and Community Development,28
5,NYC Department of Sanitation,16
6,Non-public,12
7,NYS Department of Health,8
8,Port Authority of New York and New Jersey,8
9,NYC Department of Consumer Affairs,4


Possible dups sent by different agencies

In [39]:
cross_agency = pd.read_sql_query('''SELECT f1.id, f1.facname, f2.id, f2.facname
FROM dcp.facilities f1, dcp.facilities f2
WHERE f1.bbl = f2.bbl
AND f1.geom = f2.geom
AND f1.address = f2.address
AND f1.facdomain = f2.facdomain
AND f1.facgroup = f2.facgroup
AND f1.facsubgrp = f2.facsubgrp
AND f1.factype = f2.factype
AND f1.facname <> f2.facname
AND ABS(f1.id - f2.id) = 1;''', cnx)
cross_agency

,id,facname,id,facname
0,1281,ACMH - HH CM,1282,ACMH - HH NONMED CM
1,1282,ACMH - HH NONMED CM,1281,ACMH - HH CM
2,1354,ADMIN FOR CHILDRENS SERVICE,1355,ADMINISTRATION CHILDREN SVS
3,1263,ACHIEVEMENT FIRST CHARTER SCHOOL 10,1264,ACHIEVEMENT FIRST CHARTER SCHOOL 11
4,1264,ACHIEVEMENT FIRST CHARTER SCHOOL 11,1263,ACHIEVEMENT FIRST CHARTER SCHOOL 10
...,...,...,...,...
449,32098,WEST MIDTOWN MANAGEMENT GROUP OTP,32099,WEST MIDTOWN MANAGEMENT GROUP OTP 1
450,32099,WEST MIDTOWN MANAGEMENT GROUP OTP 1,32098,WEST MIDTOWN MANAGEMENT GROUP OTP
451,32170,WESTON UNITED COMMUNITY RENEWAL - HH NONMED CM,32169,WESTON UNITED COMMUNITY RENEWAL - HH CM
452,33054,ZETA CHARTER SCHOOL - NYC 3,33055,ZETA CHARTER SCHOOL - NYC 4
